In [1]:
%config IPCompleter.greedy = True
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
%load_ext tensorboard

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import tensorflow as tf
from datetime import datetime

pd.set_option('mode.chained_assignment', None)
sn.set(rc={'figure.figsize':(9,9)})
sn.set(font_scale=1.4)

# make results reproducible
seed = 0
np.random.seed(seed)

!pip install pydot
!rm -rf ./logs/fit

# Estimators

TensorFlow Estimators are a high-level representation of a complete model, which has been designed for easy scaling and asynchronous training. However it is recommended to use the Keras API instead, which has greater flexibility in creating a model, however we include estimators for completeness.

An estimator encapsulates training, evaluation, prediction and export for serving. They have the benefit that the can easily be run on a local host machine or on a distributed multi-server environment, with CPU's, GPU's or TPU's without changing the model. They provide a safe distributed training loop that controls loading data, handling exceptions, creating model checkpoints to recover from failures and saving summaries for TensorBoard. When we work with estimators, we must separate the data input pipeline from the model.

## Pre-made Estimators

TensorFlow broadly supports the following estimators (from `tf.estimator`'s) for Classification
* `BoostedTreesClassifier` : Boosted Trees model
* `LinearClassifier` : Linear model
* `DNNClassifier` : Deep neural network model
* `DNNLinearCombinedClassifier` : Deep neural network and linear joined model 

And similarly for regression
* `BoostedTreesRegressor` : Boosted Trees model
* `LinearRegressor` : Linear model
* `DNNRegressor` : Deep neural network model
* `DNNLinearCombinedRegressor` : Deep neural network and linear joined model 

## Automatic data pre-processing

One of the benefits that Estimators have is that we specify what data type our features are and it will correctly pre-process the features for that model, i.e `numeric`, `categorical` or `string` based data. As with Keras we would have to pre-process our features before passing them to the model. We often want to turn `categorical` features into an encoding as *one-hot* representation and use an embedding to map `strings` either categorical representation (*one-hot* encodings) or some other representation such as numeric vector.

[Some](https://www.tensorflow.org/api_docs/python/tf/feature_column) of the datatypes, `tf.feature_column` that we can specify are:
* `numeric_column` : numeric features
* `categorical_column_with_vocabulary_list` : `CategoricalColumn` with a vocabulary list
* `embedding_column` : converts sparse input to categorical input
* `bucketized_column` : discretized dense input bucketed by `boundaries`

## Setup

To use estimators we have to pass a dataset loading function during training, evaluation and predicting. This function must return a tuple of two objects:
* Dictionary, of keys of *feature_name*'s, with corresponding values of *feature_tensor*'s
* Targets, tensor

For example on our digits dataset for classification

In [2]:
# Loading our digits dataset
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
digits = datasets.load_digits()

(X, y) = datasets.load_digits(return_X_y=True)
X = X.astype(np.float32)
y = y.astype(np.int32)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Pre process the data
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create the datset loading functions
def input_train_fn():
    return {'X': X_train}, y_train

def input_test_fn():
    return {'X': X_test}, y_test

We have to specify to the estimator what type of data is each feature is using the `tf.feature_column` types. In this example we have numeric data

In [3]:
feature_columns = [tf.feature_column.numeric_column('X', shape=[64])]

We can instantiate the estimator, here we will use the `tf.estimator.DNNClassifier` for our classification problem.


For the `tf.estimator.DNNClassifier` [some](https://www.tensorflow.org/api_docs/python/tf/estimator/DNNClassifier) of the keyword arguments it takes are:
* `hidden_units` : List of number of hidden units for each fully connected layer
* `feature_columns` : List of all of the `tf.feature_column` types for the input features
* `n_classes` : Number of classes
* `optimizer` : Instance of `tf.keras.optimizer.*` used to train the model, can also be a built-in string
* `activation_function` : Activation function to use for each layer, defaults to `tf.nn.relu`
* `dropuout` : Probability to drop a given neuron during training, defaults to None
* `batch_norm` : Whether to use batch normalization after each hidden layer

In [4]:
# Specify how many layers and what hidden units they have are
hidden_units = [64, 64, 64, 10]

# Number of classes
n_classes = 10

# Create the estimator
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=hidden_units,
    n_classes=n_classes,
    optimizer='Adam',
    dropout=0.1,
    batch_norm=True)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/mg/rx49509d49q70jr6f72nnrbc0000gn/T/tmpvyvd2us3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


We can then train our model passing the function to load the training dataset in the expected format. Note here we don't have epochs instead we have steps, which is how many steps to train the model for. Here we can approximate epochs with the following

In [5]:
epochs = 10
steps = epochs * X_train.shape[0]

classifier.train(input_fn=input_train_fn,
                steps=steps)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/mg/rx49509d49q70jr6f72nnrbc0000gn/T/tmpvyvd2us3/model.ckpt.
INFO:tensorflow:loss = 2.7013602, step = 0
INFO:tensorflow:global_step/sec: 173.027
INFO:tensorflow:loss = 0.7558022, step = 100 (0.579 sec)
INFO:tensorflow:global_step/sec: 251.967
INFO:tensorflow:loss = 0.35988194, step = 200 (0.397 sec)
INFO:tensorflow:global_step/sec: 244.007
INFO:tensorflow:loss = 0.20036392, step = 300 (0.410 sec)
INFO:tensorflow:global_step/sec: 246.053
INFO:tensorflow:loss = 0.1381905

INFO:tensorflow:global_step/sec: 242.767
INFO:tensorflow:loss = 0.0025985076, step = 7100 (0.412 sec)
INFO:tensorflow:global_step/sec: 233.783
INFO:tensorflow:loss = 0.010720789, step = 7200 (0.428 sec)
INFO:tensorflow:global_step/sec: 227.634
INFO:tensorflow:loss = 0.005264971, step = 7300 (0.440 sec)
INFO:tensorflow:global_step/sec: 244.609
INFO:tensorflow:loss = 0.0034968024, step = 7400 (0.408 sec)
INFO:tensorflow:global_step/sec: 252.002
INFO:tensorflow:loss = 0.011892931, step = 7500 (0.397 sec)
INFO:tensorflow:global_step/sec: 251.644
INFO:tensorflow:loss = 0.005103878, step = 7600 (0.397 sec)
INFO:tensorflow:global_step/sec: 241.595
INFO:tensorflow:loss = 0.010251191, step = 7700 (0.414 sec)
INFO:tensorflow:global_step/sec: 250.659
INFO:tensorflow:loss = 0.0074738627, step = 7800 (0.399 sec)
INFO:tensorflow:global_step/sec: 243.337
INFO:tensorflow:loss = 0.009060824, step = 7900 (0.411 sec)
INFO:tensorflow:global_step/sec: 243.402
INFO:tensorflow:loss = 0.009328728, step = 8000

Can then pass a function to load our test dataset, and evaluate our model, averaging the evaluation metrics

In [6]:
eval_result = classifier.evaluate(input_fn=input_test_fn, steps=100)
print(eval_result)
print('\nTest accuracy: {:.3%}'.format(eval_result['accuracy']))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-04-27T02:39:31Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/mg/rx49509d49q70jr6f72nnrbc0000gn/T/tmpvyvd2us3/model.ckpt-14370
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Inference Time : 0.37566s
INFO:tensorflow:Finished evaluation at 2020-04-27-02:39:31
INFO:tensorflow:Saving dict for global step 14370: accuracy = 0.98333335, average_loss = 0.31491718, global_step = 14370, loss = 0.31491706
INFO:tensorflow:Saving 'c

We can see we have somewhat easily trained a simple classification model with a sufficient test accuracy.

Can easily make predictions using our trained model as well using the `tf.estimator.DNNClassifier.predict()`

# TensorBoard

Is TensorFlow's visualization toolkit, that automatically installs when we installed TensorFlow. It is commonly used to visualize:
* metrics such as loss and accuracy during training
* the model graph (operations and layers)
* histograms of weights, biases and other tensors that change over time
* projecting embeddings to a lower dimensional space to see the structure
* profiling tensorflow models and programs
* view the best hyperparameters over a search in a parallel co-ordinate view
* and more features continually added (What-If-Tool, Fairness indicators etc)

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/fit

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Lets use an example dataset, here the MNIST dataset to create a simple model in Keras which we can visualize

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

To save TensorBoard viewable logs, we need to pass the callback `tf.keras.callbacks.TensorBoard` to our `Model.fit()` in Keras. We can also enable histogram saving every epoch with `histogram_freq=1`.

It is common practice to put the logs in a timestamped subdirectory to allow easy selection of different training runs.

In [4]:
import datetime

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 5s 76us/sample - loss: 0.2218 - accuracy: 0.9343 - val_loss: 0.1072 - val_accuracy: 0.9666
Epoch 2/5
60000/60000 [==============================] - 4s 72us/sample - loss: 0.0976 - accuracy: 0.9705 - val_loss: 0.0827 - val_accuracy: 0.9728
Epoch 3/5
60000/60000 [==============================] - 4s 74us/sample - loss: 0.0698 - accuracy: 0.9775 - val_loss: 0.0837 - val_accuracy: 0.9755
Epoch 4/5
60000/60000 [==============================] - 4s 75us/sample - loss: 0.0531 - accuracy: 0.9826 - val_loss: 0.0709 - val_accuracy: 0.9795
Epoch 5/5
60000/60000 [==============================] - 5s 77us/sample - loss: 0.0429 - accuracy: 0.9857 - val_loss: 0.0604 - val_accuracy: 0.9827


We can start TensorBoard inside the notebook or from the command line, therefore we run for:
* Notebook: `%tensorboard --logdir logs/fit`
* Command Line: `tensorboard --logdir logs/fit`

In [5]:
%tensorboard --logdir logs/fit

We can see in the dashboards above (in the above tabs):
* The **Scalars** tab plots the metrics we defined when we compiled our model, alongside any validation values if we have a validation dataset we are training with. This can also track any other scalar, such as learning rates etc.
* The **Graphs** tab visualizes our model. Which helps to check that we constructed it correctly.
* The **Distributions** and **Histograms** tabs show the distribution for a tensor for every epoch. This can be useful to verify to see that the weights and biases are changing the expected way over each epoch.

Also extra TensorBoard plugins are automatically enabled when we log other types of data as well. For example, the [Keras TensorBoard callback](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard) lets us log images and embeddings as well. We can see what other plugins that TensorBoard supports that are currently inactive by the tab dropdown in the top right.